In [10]:
import requests
import os
from xml.etree import ElementTree 
import pandas

from typing import Dict
from typing import Tuple
from typing import Optional
from typing import List


host_name = "www.deribit.com"
deribit_endpoint_v2 = f"https://{host_name}/api/v2"
public_url = f"{deribit_endpoint_v2}/public"
private_url = f"{deribit_endpoint_v2}/private"



def authorize_with_credentials(client_id: str, client_secret: str) -> Tuple[Dict[str, str], str]:
    print(f"connecting with {client_id}")
    headers = {"Content-Type": "application/json"}
    auth_params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials"
    }
    authentication = requests.get(f"{public_url}/auth", params=auth_params, headers=headers).json()
    headers["Authorization"] = f"Bearer {authentication['result']['access_token']}"
    refresh_token = authentication['result']['refresh_token']
    return headers, refresh_token

def authorize_refresh(refresh_token: str) -> Tuple[Dict[str, str], str]:
    print(f"refreshing authentication {refresh_token}")
    headers = {"Content-Type": "application/json"}
    auth_params = {
        "refresh_token": refresh_token,
        "grant_type": "refresh_token"
    }
    authentication = requests.get(f"{public_url}/auth", params=auth_params, headers=headers).json()
    headers["Authorization"] = f"Bearer {authentication['result']['access_token']}"
    new_refresh_token = authentication['result']['refresh_token']
    return headers, new_refresh_token

def run_get(target: str, params: Dict[str, str], headers: Dict[str, str], refresh_token: str=None) -> Optional[Tuple[Dict[str, str], str]]:
    response = requests.get(target, params=params, headers=headers)
    if response.status_code >= 399:
        error = response.json()
        if error['error']['code'] == 13009 and refresh_token:
            # re-run authorization
            new_headers, refresh_token = authorize_refresh(refresh_token)
            response = requests.get(target, params, new_headers)
        else:
            return None

    return response.json()['result'], headers, refresh_token


class RowData:
    def __init__(self, row: Dict[str, str]):
        self._row = row

    def __repr__(self):
        return repr(self._row)


def load_ib_xml_report(ib_token: str, ib_query_id: str) -> ElementTree:
    ib_flex_url = f"https://www.interactivebrokers.com/Universal/servlet/FlexStatementService.SendRequest?t={ib_token}&q={ib_query_id}&v=3"
    ib_response = requests.get(ib_flex_url)
    ib_report_location = ElementTree.fromstring(ib_response.content)
    ib_report_reference_code = ib_report_location.find('ReferenceCode').text
    ib_report_base_url = ib_report_location.find('Url').text
    ib_report_url = f"{ib_report_base_url}?t={ib_token}&q={ib_report_reference_code}&v=3"
    ib_report = requests.get(ib_report_url)
    return ElementTree.fromstring(ib_report.content)

def check_report_error(report_tree: ElementTree) -> Optional[int]:
    error_node = report_tree.find('.//ErrorCode')
    if error_node:
        return int(error_node.text)
    
    return None

def make_ib_records(report_tree: ElementTree) -> List[RowData]:
    ib_records = []
    for position in report_tree.findall('.//OpenPosition'):
        ib_records.append(RowData(position.attrib))
    return ib_records


In [ ]:

headers, refresh_token = authorize_with_credentials(client_id=os.getenv("DERIBIT_CLIENT_ID"), client_secret=os.getenv("DERIBIT_CLIENT_SECRET"))

account_summary_deribit = {}
for currency in ("BTC", "ETH", "USDT", "USDC"):
    account_summary_params = {"currency": currency}
    result, headers, refresh_token = run_get(f"{private_url}/get_account_summary", params=account_summary_params, headers=headers, refresh_token=refresh_token)
    account_summary_deribit[currency] = result

account_summary_deribit


In [4]:
ib_token = os.getenv("IB_FLEX_REPORT_TOKEN")
ib_query_id = os.getenv("IB_FLEX_QUERY_ID")
ib_report = load_ib_xml_report(ib_token, ib_query_id)
error = check_report_error(ib_report)
if error:
    print(f'report in error: {error}, try again later')
    raise ValueError(f'report in error: {error}, try again later')
ib_positions = make_ib_records(ib_report)
pandas.DataFrame(ib_positions)


""


In [12]:
ElementTree.tostring(ib_report)
ib_report.find('.//ErrorCode').text


'1019'